# Landsat Multi-Temporal Composites

In remote sensing, we usually work on single images ("scenes") acquired on a given date over a given area. While this sufficient or even necessary for many applications such as the studying of vegetation dynamics, some applications actually require information combined from many scenes, i.e., multiple "snapshots" of an area over time.

Single scenes tend to be noisy mostly because of atmospheric disturbances and instrumentation errors. Thus single scenes might contain significant parts of no-data or poor-quality pixels due to instrument failures, clouds (in the case of optical imagery) or shadows. Taking multiple scenes acquired at different points in time and aggregating them over time by a statistical measure such as the mean or median is therefore in some cases the preferred choice.

Multi-temporal composites are used for many different applications, including

* inputs for classification tasks such as land-use/ land-cover mapping
* generation of equidistant time series
* generation of basemaps

to name a few.

In this notebook, you will learn how to generate a multi-temporal mosaic of Landsat8-9 scenes acquired in Spring 2022. Our example covers a full month (May 2022). The steps are:

1. We fetch the Landsat scenes overlapping our study region
2. We aggregate the Landsat scenes into a single multi-temporal composite using the median, i.e., our composite will contain the median reflectance obtained from all Landsat scenes fetched in May 2022


In [ ]:
# we have to install the development version for the time being
#!pip uninstall eodal -y
#!pip install git+https://github.com/lukasValentin/eodal@landsat-dev

## Setting up the EOdal Mapper

In [3]:
years = [1984, 1986, 1987, 1989, 1990, 1994, 1995, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

In [4]:
from datetime import datetime
from pathlib import Path
from shapely.geometry import box
from matplotlib import pyplot as plt
import numpy as np

from eodal.config import get_settings
from eodal.core.sensors import Landsat
from eodal.mapper.feature import Feature
from eodal.mapper.filter import Filter
from eodal.mapper.mapper import Mapper, MapperConfigs
from eodal.core.band import GeoInfo, Band
from eodal.core.raster import RasterCollection

In [6]:
# define a function to mask clouds and shadows
def preprocess_landsat_scene(
        ds: Landsat
) -> Landsat:
    """
    Mask clouds and cloud shadows in a Landsat scene based
    on the 'qa_pixel' band.

    NOTE:
        Depending on your needs, the pre-processing function can be
        fully customized using the full power of EOdal and its
        interfacing libraries!

    :param ds:
        Landsat scene before cloud mask applied.
    :return:
        Landsat scene with clouds and cloud shadows masked.
    """
    ds.mask_clouds_and_shadows(inplace=True)
    return ds

In [7]:
Settings = get_settings()
# we use STAC, i.e., Microsoft Planetary Computer
Settings.USE_STAC = True

# user-inputs
# -------------------------- Collection -------------------------------
collection = 'landsat-c2-l2'

# ---------------------- Spatial Feature  ------------------------------
# can be also shp, gpkg, etc.

#bbox = box(*[30.2825, 0.4019, 30.3714, 0.4643]) # 1. Area
bbox = box(*[30.2703, 0.4043, 30.4482, 0.5291]) # 2. Area
feature = Feature(
    name='landsat-composite',
    geometry=bbox,
    epsg=4326,
    attributes={})

# ------------------------- Metadata Filters ---------------------------
metadata_filters = [
    Filter('eo:cloud_cover', '<', 30),
    Filter('landsat:wrs_path', '==', '173'),
    Filter('landsat:wrs_row', '==', '060'),
    #Filter('instruments', '!=', 'etm+')
]

for year in years: 
    
    # ------------------------- Time Range ---------------------------------
    time_start = datetime(year, 1, 1)
    time_end = datetime(year, 12, 31)

    # set up the Mapper configuration
    mapper_configs = MapperConfigs(
        metadata_filters=metadata_filters,
        collection=collection,
        feature=feature,
        time_start=time_start,
        time_end=time_end)

    # get a new mapper instance
    mapper = Mapper(mapper_configs)

    # fetch the metadata
    # query the scenes available (no I/O of scenes, this only fetches metadata)
    mapper.query_scenes()
    print(str(year), f'Number of Landsat scenes found: {mapper.metadata.shape[0]}')

    # define e bands to read in 'band_selection' and to preprocess the scenes
    scene_kwargs = {
        'scene_constructor': Landsat.from_usgs,
        'scene_constructor_kwargs': {
            'band_selection': ['blue', 'green', 'red', 'nir08', 'swir16', 'qa_pixel'], 
            'read_qa': False},
        'scene_modifier': preprocess_landsat_scene,
        'scene_modifier_kwargs': {}}

    # now we load the scenes
    mapper.load_scenes(scene_kwargs=scene_kwargs)

    # The mapper returns the single scenes. 
    # As we told the EOdal to mask out clouds, a significant share of the pixels is masked out. 
    # We will aggregate them in the next step.
    #f = mapper.data.plot(band_selection=['red', 'green', 'blue'], figsize = (20, 20), max_scenes_in_row = 2)

    #save as PNG for quick view
    #f.savefig('S:\MSc_23_TimckeFinn\data\python_outputs\landsat_scenes_' + str(year) + '.png')
    #First, we open masked arrays for storing the data:

    # all scenes have the same shape, i.e., the same number of bands, rows and columns
    shapes = [{timestamp: scene.get_values().shape} for timestamp, scene in mapper.data]

    # open arrays for storing the data per band
    shape = (len(mapper.data), list(shapes[0].values())[0][1], list(shapes[0].values())[0][2])
    blue = np.ma.masked_array(data=np.ndarray(shape, dtype=float), mask=False)
    red = np.zeros_like(blue)
    green = np.zeros_like(blue)
    nir08 = np.zeros_like(blue)
    swir16 = np.zeros_like(blue)

    # Next, we loop over the scenes.
    idx = 0
    for _, scene in mapper.data:
        blue[idx, :, :] = scene['blue'].values
        red[idx, :, :] = scene['red'].values
        green[idx, :, :] = scene['green'].values
        nir08[idx, :, :] = scene['nir08'].values
        swir16[idx, :, :] = scene['swir16'].values
        idx += 1

    # Finally, we aggregate the data using the median reflectance
    # calculate the median reflectance per spectral band
    blue_median = np.ma.median(blue, axis=0)
    green_median = np.ma.median(green, axis=0)
    red_median = np.ma.median(red, axis=0)
    nir08_median = np.ma.median(nir08, axis=0)
    swir16_median = np.ma.median(swir16, axis=0)

    rc = RasterCollection()

    bands = {'blue': blue_median, 
            'green': green_median, 
            'red': red_median, 
            'nir08' : nir08_median, 
            'swir16' : swir16_median}

    for band_name, band_value in bands.items():
        rc.add_band(
            band_constructor=Band,
            values=band_value,
            band_name=f'{band_name}_median',
            geo_info=scene[band_name].geo_info)
        
    # save as GeoTiff for further analysis
    #rc.to_rasterio('S:\MSc_23_TimckeFinn\data\EOdal\landsat_median_composite_' + str(year) + '.tif')


2023-06-22 16:20:31,423 eodal        INFO     Starting extraction of landsat scenes


1984 Number of Landsat scenes found: 1


2023-06-22 16:20:35,650 eodal        INFO     Finished extraction of landsat scenes
2023-06-22 16:20:36,536 eodal        INFO     Starting extraction of landsat scenes


1986 Number of Landsat scenes found: 2


2023-06-22 16:20:42,647 eodal        INFO     Finished extraction of landsat scenes
2023-06-22 16:20:43,725 eodal        INFO     Starting extraction of landsat scenes


...

2023-06-22 16:22:31,638 eodal        INFO     Finished extraction of landsat scenes
2023-06-22 16:22:33,393 eodal        INFO     Starting extraction of landsat scenes


2004 Number of Landsat scenes found: 4


2023-06-22 16:22:49,662 eodal        INFO     Finished extraction of landsat scenes
2023-06-22 16:22:51,833 eodal        INFO     Starting extraction of landsat scenes


2005 Number of Landsat scenes found: 7


KeyboardInterrupt: 

In [ ]:
# # We plot the result
# f, ax = plt.subplots(figsize = (20, 10))
# f = rc.plot_multiple_bands(['red_median', 'green_median', 'blue_median'], ax=ax)

# #save as PNG for quick view
# f.savefig('S:\MSc_23_TimckeFinn\data\python_outputs\landsat_median_composite_' + str(year) + '.png')
